In [1]:
%matplotlib inline

import numpy as np
import random
import pandas as pd
import requests as rq
import sys
import io
import os
from bs4 import BeautifulSoup
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.optimizers import RMSprop
from collections import Counter
from datetime import datetime
import keras
import keras.callbacks
from keras.callbacks import TensorBoard
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from keras.layers import LSTM, Dense, Dropout
from keras.callbacks import Callback


%load_ext tensorboard

2024-04-11 16:27:25.706151: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-11 16:27:25.733303: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Data Cleaning

In [4]:

##get data
##data is already from 2012-2020
data = pd.read_csv('cleaneddata.csv') 


##preprocessing

##filter to only patriots
data = data.loc[(data['home_team'] == 'NE') | (data['away_team'] == 'NE')]

##removes 
data = data.loc[(data['desc'].str.startswith('('))]
data = data[data['play_type'] != 'qb_kneel']
data = data[data['play_type'] != 'no_play']




##fixes playtype to be strictly pass or run 
##lables scrambles as passes
data.loc[data['pass']==1,'play_type'] = 'pass'
data.loc[data.rush==1,'play_type'] = 'run'


columnsKeep = ['play_id','game_id','season_type','week','posteam','posteam_type','defteam','side_of_field','yardline_100','half_seconds_remaining','home_team','away_team',
               'game_seconds_remaining','game_half','drive','down','time','yrdln','ydstogo','desc','play_type','yards_gained','shotgun','no_huddle','qb_dropback','qb_kneel',
               'qb_spike','qb_scramble','pass_length','run_location','run_gap','home_timeouts_remaining','away_timeouts_remaining','total_home_score','total_away_score',
               'passer_player_name','receiver_player_name','rusher_player_name','play_type_nfl','roof','surface','temp','wind','home_coach','away_coach','game_stadium',
               'success']

data = data[columnsKeep]


data.to_csv('cleaneddata.csv', index=False)


/tmp/ipykernel_56490/2276483710.py:3: DtypeWarning: Columns (44,45,179,180,189,190,195,196,203,204,218,219,220,222,224,226,253,254,255,260,262,263,283,284,294,295) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('cleaneddata.csv')


In [38]:
features = data[['play_id','game_id','season_type','week','posteam','posteam_type','defteam','side_of_field','yardline_100','half_seconds_remaining','home_team','away_team',
               'game_seconds_remaining','game_half','drive','down','time','yrdln','ydstogo','desc','play_type','yards_gained','shotgun','no_huddle','qb_dropback','qb_kneel',
               'qb_spike','qb_scramble','pass_length','run_location','run_gap','home_timeouts_remaining','away_timeouts_remaining','total_home_score','total_away_score',
               'passer_player_name','receiver_player_name','rusher_player_name','play_type_nfl','roof','surface','temp','wind','home_coach','away_coach','game_stadium',
               'success']
]
target = data['play_type'].map({'PASS': 0, 'RUSH': 1})

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['down','ydstogo','yardline_100', 'half_seconds_remaining', 'game_seconds_remaining', 'temp', 'wind'])
    ]
)

X_processed = preprocessor.fit_transform(features)

X_lstm = X_processed.reshape((X_processed.shape[0], 1, X_processed.shape[1]))
y = target.values

X_train, X_test, y_train, y_test = train_test_split(X_lstm, y, test_size=0.2, random_state=42)

model = Sequential([
    LSTM(128, input_shape=(X_train.shape[1], X_train.shape[2])),
    Dense(1, activation='softmax')
])

model.compile(
    loss='binary_crossentropy',
    optimizer=RMSprop(learning_rate=0.01),
    metrics=['accuracy']
)



##history = model.fit(X_train, y_train, epochs=50, validation_split=0.1, callbacks=[early_stopping], verbose=1)

##test_loss, test_acc = model.evaluate(X_test, y_test, verbose=1)
##print(f'Test Accuracy: {test_acc}')



# Assuming you have already defined X_test and y_test
predictions = model.predict(X_test)
predictions = (predictions > 0.5).astype(int).reshape(-1)

early_stopping = EarlyStopping(monitor='val_loss', patience=10)
history = model.fit(X_train, y_train, epochs=50, validation_split=0.1, callbacks=[early_stopping], verbose=1)

test_loss, test_acc = model.evaluate(X_test, y_test, verbose=1)
print(f'Test Accuracy: {test_acc}')


for i in range(10):
    actual = 'PASS' if y_test[i] == 0 else 'RUSH'
    predicted = 'PASS' if predictions[i] == 0 else 'RUSH'
    print(f'Actual: {actual}, Predicted: {predicted}')


147/147 [==============================] - 0s 293us/step


2024-04-11 00:32:40.590447: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-04-11 00:32:40.591221: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-04-11 00:32:40.591578: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Epoch 1/50


2024-04-11 00:32:40.875937: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-04-11 00:32:40.876641: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-04-11 00:32:40.877132: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

526/526 [==============================] - 1s 1ms/step - loss: nan - accuracy: 0.5122 - val_loss: nan - val_accuracy: 0.5321
Epoch 2/50
139/526 [======>.......................] - ETA: 0s - loss: nan - accuracy: 0.5135

2024-04-11 00:32:41.867369: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-04-11 00:32:41.867826: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-04-11 00:32:41.868401: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

526/526 [==============================] - 0s 826us/step - loss: nan - accuracy: 0.5125 - val_loss: nan - val_accuracy: 0.5321
Epoch 3/50
526/526 [==============================] - 0s 815us/step - loss: nan - accuracy: 0.5125 - val_loss: nan - val_accuracy: 0.5321
Epoch 4/50
526/526 [==============================] - 0s 796us/step - loss: nan - accuracy: 0.5125 - val_loss: nan - val_accuracy: 0.5321
Epoch 5/50
526/526 [==============================] - 0s 764us/step - loss: nan - accuracy: 0.5125 - val_loss: nan - val_accuracy: 0.5321
Epoch 6/50
526/526 [==============================] - 0s 765us/step - loss: nan - accuracy: 0.5125 - val_loss: nan - val_accuracy: 0.5321
Epoch 7/50
526/526 [==============================] - 0s 763us/step - loss: nan - accuracy: 0.5125 - val_loss: nan - val_accuracy: 0.5321
Epoch 8/50
526/526 [==============================] - 0s 765us/step - loss: nan - accuracy: 0.5125 - val_loss: nan - val_accuracy: 0.5321
Epoch 9/50
526/526 [=========================